In [ ]:
import pandas as pd
df1 = pd.read_csv('/Users/junwlee/Documents/KoreaUniv/24-Spring/Statistics&ML/practice/Chapter2/rosman-store-sales/train.csv', engine='python')
print(df1.head(10))

In [ ]:
df2 = pd.read_csv('/Users/junwlee/Documents/KoreaUniv/24-Spring/Statistics&ML/practice/Chapter2/rosman-store-sales/store.csv', engine='python')
print(df2.head(10))

In [ ]:
df = pd.merge(df1, df2, on='Store')
print(df, df.shape)
print(df.dtypes)

In [ ]:
print(len(df['Store'].unique()))
print(len(df['Date'].unique()))
print(df['DayOfWeek'].value_counts())

In [ ]:
import numpy as np

df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df['Month'] = df['Date'].dt.month
df['Quarter'] = df['Date'].dt.quarter
df['Year'] = df['Date'].dt.year
df['Day'] = df['Date'].dt.day
df['Week'] = df['Date'].dt.isocalendar().week
df['Season'] = np.where( df['Month'].isin([3,4,5]),'Spring',
                         np.where( df['Month'].isin([6,7,8]), 'Summer',
                            np.where( df['Month'].isin([9,10,11]), 'Fall',
                                np.where( df['Month'].isin([12,1,2]), 'Winter','None'))))
print(df[['Date','Year', 'Month', 'Day','Week','Quarter', 'Season']])

In [ ]:
print(df)

In [ ]:
df.hist(figsize=(20,10))

In [ ]:
print(df.isnull().sum() / df.shape[0] * 100)

In [ ]:
df['CompetitionDistance'] = df['CompetitionDistance'].fillna(df['CompetitionDistance'].mode()[0])
print(df['CompetitionDistance'].isnull().sum())

# 범주형 자료의 변환

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 결측치가 많은 자료를 제외한 변수
n_columns = ['Customers', 'Open', 'Promo', 'Promo2', 'StateHoliday', 'SchoolHoliday', 'CompetitionDistance']

categ_columns = ['DayOfWeek', 'Quarter', 'Month', 'Year', 'StoreType', 'Assortment', 'Season']

In [ ]:
def dummy(data, col):
    lab=LabelEncoder() # 범주형 자료를 수치화
    aa=lab.fit_transform(data[col]).reshape(-1,1)
    one=OneHotEncoder(sparse_output=False) # 가변수화
    column_names = [col + '_' + str(i) for i in lab.classes_]
    return pd.DataFrame(one.fit_transform(aa), columns=column_names)

# classes_ : Holds the label for each class.

# LabelEncoder
- 예를 들어, Season 변수가 Spring, Summer, Fall, Winter의 4가지 값을 가진다면, 이들은 각각 0, 1, 2, 3과 같은 정수로 변환

# reshape :
- LabelEncoder 같은 일부 변환기는 범주형 데이터를 변환할 때 1차원 배열(예: (샘플 수,))을 반환
- oneHotEncoder와 같은 다른 변환기를 적용하기 전에 입력 데이터가 2차원 형태가 되어야 하는데, 여기서 1차원 배열을 2차원 배열로 만들기 위해 reshape(-1, 1) 과정이 필요

# OneHotEncoder
- 예를 들어, Season이 Spring, Summer, Fall, Winter의 값을 가진다면, 이는 Season_Spring, Season_Summer, Season_Fall, Season_Winter라는 4개의 열로 변환

# dummy( ):
- 입력된 데이터프레임(data)의 지정된 열(col)에 대해 수치화와 가변수화를 수행하고, 그 결과를 새로운 데이터프레임으로 반환
- 이렇게 변환된 데이터프레임은 모델 학습에 적합한 형태로 데이터를 변환하여, 머신러닝 모델이 범주형 데이터를 효율적으로 처리

In [ ]:
fdata = df[n_columns]
print(fdata.head())

In [ ]:
for column in categ_columns:
    # 지정된 모든 범주형 열(categ_columns)에 대해 변환 작업을 수행
    temp_df = dummy(df, column)
    
    # 이를 기존의 수치형 데이터(fdata)와 합쳐 최종적으로 모델 학습에 사용될 데이터셋을 준비
    fdata = pd.concat([fdata, temp_df], axis=1)
fdata.head()

In [ ]:
fdata.shape

In [ ]:
fdata.dtypes.unique()

In [ ]:
fdata['StateHoliday'].unique()

In [ ]:
fdata['StateHoliday'] = np.where(fdata['StateHoliday'] == 'o', 0, 1)

# np.where
# np.where 함수는 세 개의 주요 파라미터를 받는다: 조건문, 참일 때의 값, 거짓일 때의 값
# 조건문은 fdata['StateHoliday'] == 'o'로, 'StateHoliday' 열의 값이 'o'와 같은지를 검사
# 조건이 참이면 0을, 거짓이면 1을 해당 위치에 할당